# Phase Plate simulation for FFTW
Preparing options using FFTW

In [1]:
# –––– Standard Imports that don't need to be reloaded for testing –––––––––
from PhasePlateAWA import Gaussian, MaskOptics, Propagate, Lens
import numpy as np
from numpy import pi
import yaml
import matplotlib.pyplot as plt
from Targets import flatTop, superTruncGaussian
from PlottingTools import StanfordColormap, plotBeam, plotBeamFlatTop, AutoCorr
import ast
import h5py
from cmocean import cm as cmo
import RefractionIndex
from PhysicalPlate import maskToDepths, Density
from matplotlib.colors import ListedColormap
from cmocean import cm
from ToolsIFTA import SSE
import subprocess

stanford_colormap = StanfordColormap()


## Preparing run functions for S3DF

In [2]:
def createSetup(inputField, target, saveH5 = 'setupIFTA.h5', iteration = 30, f = 12e-1, z = 1.2,
            size = 0, plot = False, SSE = False, save = '', nProp = 1.00030067,
            steps = None, boxing = None, wavelength = 253e-9, randomSeed = 20, z0 = 1.1e-3,
            nLens = 1.5058500198911624, extent = [-1.27 * 1e-2, 1.27 * 1e-2], realLens = False):

    # ––– Save the inputs to an h5 file for retrieval in IFTA ––––––––––––––––––––
    with h5py.File(saveH5, 'w') as hf:
        # ––– Save the input field & target in datasets ––––––––
        hf.create_dataset('inputField', data = inputField)
        hf.create_dataset('target', data = target)
        hf.create_dataset('extent', data = extent)
        # ––– Save the other parameters as attributes ––––––––––
        # Main parameters
        hf.attrs['iteration'] = iteration
        hf.attrs['wavelength'] = wavelength
        hf.attrs['f'] = f
        hf.attrs['z'] = z
        hf.attrs['z0'] = z0
        hf.attrs['size'] = size
        hf.attrs['nLens'] = nLens
        hf.attrs['nProp'] = nProp
        hf.attrs['save'] = save
        hf.attrs['steps'] = steps
        # Selectables
        hf.attrs['plot'] = plot
        hf.attrs['realLens'] = realLens
        hf.attrs['SSE'] = SSE
        hf.attrs['randomSeed'] = randomSeed
        
    return 

In [ ]:
!pwd
!hostname

In [ ]:
def run_IFTA(inputField, iteration = 30, f = 1.2, z = 1.2, target = None,
            size = 0, plot = False, SSE = False, save = '', nProp = 1.00030067,
            steps = None, boxing = None, wavelength = 253e-9, randomSeed = 20, z0 = 1.1e-3,
            nLens = 1.5058500198911624, extent = [-1.27 * 1e-2, 1.27 * 1e-2], realLens = None):

    # ––– Prepare the Config file ––––––––––––––––––––––––––––––
    createSetup(inputField, iteration = iteration, target = target,
                       plot = plot, SSE = SSE, steps = steps,
                       save = save, realLens = realLens, boxing = boxing, z0 = z0,
                       f = f, z = z, randomSeed = randomSeed, nProp = nProp, nLens = nLens,
                       extent = extent, size = size, wavelength = wavelength)

    # ––– Prepare the Scratch config –––––––––––––––––––––––––––
    nproc = 40
    PARTITION = 'PARTITION'
    ACCOUNT = 'ACCOUNT'
    JOBNAME = 'PhasePlate'
    

    # ––– Prepare the IFTA in Scratch Node –––––––––––––––––––––
    cmdPrep = (
            "bash -c \""
              "hostname && cd && pwd && "
              "scp /CODE_FOLDER/IFTA_FFTW.py "
              "/RUN_FOLDER/ && "
              "scp /CODE_FOLDER/setupIFTA.h5 "
              "/RUN_FOLDER/ && "
              "scp /CODE_FOLDER/ToolsIFTA.py "
              "/RUN_FOLDER/ && "
              "scp /CODE_FOLDER/padding.py "
              "/RUN_FOLDER/ &&"
              "scp /CODE_FOLDER/Targets.py "
              "/RUN_FOLDER/ &&"
              "scp /CODE_FOLDER/PhysicalPlate.py "
              "/RUN_FOLDER/ &&"
              "scp /CODE_FOLDER/PlottingTools.py "
              "/RUN_FOLDER/ "
            "\"" 
            )
    try:
        result = subprocess.run(
            cmdPrep, shell=True,
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            text=True, check=True
        )
        print("=== STDOUT ===\n", result.stdout)
        print("Completed Setup")
    except subprocess.CalledProcessError as e:
        print("=== RETURN CODE ===", e.returncode)
        print("=== STDOUT ===\n", e.stdout)
        print("=== STDERR ===\n", e.stderr)
        raise

    # ––– Run the IFTA through the Scratch Nodes –––––––––––––––
    cmdRun = (
            f"salloc --partition={PARTITION} --account={ACCOUNT} "
            f"--nodes=1 --job-name={JOBNAME} --ntasks=1 --cpus-per-task=40 "
            "bash -c \""
              "cd && cd /RUN_FOLDER/ && pwd && python -u IFTA_FFTW.py "
            "\"" 
            )
    print(f'Running : {cmdRun}')

    ret = subprocess.run(cmdRun, shell=True)  
    print("==> salloc/IFTA finished with exit code", ret.returncode)
    
    
    # ––– Retrieve the h5 output File and clean up –––––––––––––
    cmdClean = (
            "bash -c \""
              f"hostname && pwd && scp /RUN_FOLDER/{save} " 
              "/CODE_FOLDER/"
            "\"" 
            )
    
    try:
        result = subprocess.run(
            cmdClean, shell=True,
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            text=True, check=True
        )
        print("=== STDOUT ===\n", result.stdout)
        print("Completed Clean Up")
    except subprocess.CalledProcessError as e:
        print("=== RETURN CODE ===", e.returncode)
        print("=== STDOUT ===\n", e.stdout)
        print("=== STDERR ===\n", e.stderr)
        raise

    return

In [5]:
def phasePlate(inputBeam, hologram = [30, None], wavelength = 253e-9, f = 1.2,
               w0 = 4e-3, extent = [-1.27 * 1e-2, 1.27 * 1e-2], plot = False,
               realLens = True, boxing = None, steps = None, SSE = True, z0 = None,
               IFTAPlotting = True, randomSeed = 15, save = '', nProp = 1, nLens = 1,
               size = 0):
    """
    Phase Plate transfer function

    Parameters
    ----------
    inputBeam : np.array
        The input beam that should go through the phase plate
    hologram : array or string, optional
        if array -> [iterations of GSA, GSA target], 
        This will launch an IFTA phase retrieval for the target The default is [30, None].
        if string -> This is the save file of type 'filename.h5'
    wavelength : float, optional
        Single wavelength of the beam. The default is 253 nm
    w0 : float, optional
        The waist of the curve using 1/e of the amplitude. The default is 4 mm
    extent : array, optional
        Extent of the array to build. The default is set in the globals.
    plot : Bool, optional
        Boolean to choose if plots should be made. The default is False.

    Returns
    -------
    outputBeam : np.array
        meshgrid of the beam after passing through the phase plate

    """    
    
    if len(hologram) == 2:
        iterations, target = hologram
        hologram = run_IFTA(inputBeam, iteration = iterations, target = target,
                       plot = IFTAPlotting, SSE = SSE, steps = steps,
                       save = save, realLens = realLens, boxing = boxing, z0 = z0,
                       f = f, z = f, randomSeed = randomSeed, nProp = nProp, nLens = nLens,
                       extent = extent, size = size, wavelength = wavelength)
    else:
        with h5py.File(hologram, 'r') as file:
            hologram = file['Phase'][:]
    
    
    
    # --- Adding etching uncertainty ---
    '''uncertainty = np.zeros(hologram.shape)
    print(np.unique(hologram))
    randomOffset = np.random.rand(len(np.unique(hologram)))
    #print(randomOffset)
    for i, phase in enumerate(np.unique(hologram)):

        uncertainty[hologram == phase] = randomOffset[i] * 0.011175626040438 * 0
        
    uncertainty = MaskOptics(uncertainty, 1.27e-2, extent = extent)
    hologram += uncertainty'''
    
    # --- Adding slow varying uncertainty to the phase plate ---
    '''uncertainty = Uncertainty(hologram, wavelength, extent = extent, typeU = Gaussian)
    #uncertainty = (uncertainty + np.pi) % (2 * np.pi) - np.pi
    plt.imshow(uncertainty, cmap = stanford_colormap)
    plt.title('Uncertainty Thickness in Rad')
    plt.colorbar()
    plt.show()
    hologram += uncertainty
    
    hologram = (hologram + np.pi) % (2 * np.pi) - np.pi'''
    
    inputPhase = np.angle(inputBeam)

    with h5py.File(save, 'r') as file:
            hologram = file['Phase'][:]
        
    phasePlate = np.subtract(hologram, inputPhase)
    
    plt.imshow(hologram, cmap = cmo.curl)
    plt.colorbar()
    plt.title('Phase Plate')
    plt.show()
    
    outputBeam = inputBeam * np.exp(1j * phasePlate)
    
    plot = False
    if plot:
        fig, (axA, axB) = plt.subplots(2, 2, figsize=(12, 10))
        
        outphase = np.angle(outputBeam)
        subtract = axA[0].imshow(outphase, cmap = 'vlag', extent = [extent[0], extent[1], extent[0], extent[1]])
        Real = axA[1].imshow(outputBeam.real, extent = [extent[0], extent[1], extent[0], extent[1]])
        Imag = axB[0].imshow(outputBeam.imag, extent = [extent[0], extent[1], extent[0], extent[1]])
        Intensity = axB[1].imshow(np.abs(outputBeam)**2, cmap = stanford_colormap
                                  , extent = [extent[0], extent[1], extent[0], extent[1]])
        
        fig.colorbar(subtract, ax = axA[0], orientation = 'vertical')
        fig.colorbar(Real, ax = axA[1], orientation = 'vertical')
        fig.colorbar(Imag, ax = axB[0], orientation = 'vertical')
        fig.colorbar(Intensity, ax = axB[1], orientation = 'vertical')

        axA[0].set_title("Output Phase")
        axA[1].set_title("Real Part")
        axB[0].set_title("Imaginary part")
        axB[1].set_title("Intensity")
        
        axA[0].set_xlabel('Beam size (m)')
        axA[1].set_xlabel('Beam size (m)')
        axB[0].set_xlabel('Beam size (m)')
        axB[1].set_xlabel('Beam size (m)')
        axA[0].set_ylabel('Beam size (m)')
        fig.suptitle("Beam Output Characterstics Post-Phase-Plate", size = 20)
        fig.tight_layout()
        plt.show()
        
    return outputBeam

## Running the IFTA 

In [ ]:
# --- Globals ---
wavelength = 262 * 1e-9
w0 = 10*1e-3 # 1 inch diameter facet #6 * 1e-3
f = 2.7
extent = 2*np.array([-1.27 * 1e-2, 1.27 * 1e-2])
z0 = pi/wavelength * w0**2
randomSeed = 20# 30 was used before zone plate testing
np.random.seed(randomSeed) #Setting the random seed for the IFTA
nLens = RefractionIndex.FusedSilica(wavelength)  #1.5058500198911624 # None
nProp = RefractionIndex.Air(wavelength) #1.00030067 # 1
z1 = (2.2e-3) / 2 # None #Half thickness of a 1m lens
sizeFactor = 13

steps = 3
iterations = 50
# --- Save Files ---

###########################################################
# --- Creating a Phase Plate and Propagating through it --- 
###########################################################

# --- Creating a Pure Gaussian Beam to Propagate ---- 
inputBeam = Gaussian(sizeFactor = sizeFactor, plot = False, w0 = w0, extent = extent, wavelength = wavelength)
inputBeam = MaskOptics(inputBeam, 2*1.27e-2, extent = extent) #Cutting the input beam at 1 inch aperture

# --- Initializing a target ---
targetRadius = (11.1/4)*1e-3 # Radius of the target that should be achieved after propagation
#trunc = 50#69.8#50#69.8 # Truncation Percentage
#target = superTruncGaussian(inputBeam, targetRadius = targetRadius, trunc = trunc, extent = extent)
target = flatTop(inputBeam, w0 = targetRadius, extent = extent)

#plotBeam(np.sqrt(target), extent = extent, truncRadius = targetRadius, fitCut = True, title = 'Beam Profile at Cathode', maxROI = 1000)


runName = f'FlatTop_2inch_2^{sizeFactor}_f={f}m_target={targetRadius*2*1e3}mm_w0={w0*1e3}e-3_steps={steps}_iter={iterations}'
print(runName)
savePhase = f'IFTA_FFTW_{runName}.h5'#1inch_2^11_f=1m_w0=10mm_iter=1000_target=11.1mm_9levels.h5'
saveData = f'SimulatedData_{runName}.h5'#1inch_2^11_f=1m_w0=10mm_iter=1000_target=11.1mm_9levels.h5' #'IFTAPhases/Facet Aligner/SimulatedData_4level_2^11_200iter.h5'
#hologramSave = 'IFTAPhases/Density_Analysis/Phase_Nonelevels'


In [ ]:
# --- Building a phase plate to achieve the given target ---
print(f'Starting IFTA for {runName}...')
plate = phasePlate(inputBeam, plot = True, nProp = nProp, nLens = nLens, hologram = [iterations, target],
                   save = savePhase, f = f, z0 = z1, randomSeed = randomSeed, steps = steps, extent = extent,
                   size = 0, wavelength = wavelength, SSE = False)# [30,target] / hologramSave


In [ ]:
#savePhase = ""
!pwd
print(savePhase)
with h5py.File(savePhase, 'r') as file:
            outputFP = file['OutputFP'][:][-1]
            phase = file['Phase'][:]
print(np.unique(phase))


In [ ]:
plotBeam(np.sqrt(outputFP), extent = extent, truncRadius = targetRadius, fitCut = True, title = 'Beam Profile at Cathode', maxROI = 500)

In [ ]:
print(SSE(outputFP, target))

## Plotting the Phase Plate to Manufacture

In [ ]:
# --- Making a manufacturable plate with true depth levels ---

truePlate = maskToDepths(phase, save = '', wavelength=wavelength,
                         n_air = nProp, materialIndex=nLens)

figT, axsT = plt.subplots(1, 2, figsize=(10, 6))
# Plotting initial Phase
n_unique = len(np.unique(truePlate))
#cmap = ListedColormap(get_cmap('tab20').colors[:n_unique])
cmap = ListedColormap(cm.haline(np.linspace(0, 1, n_unique)))
extentPlot = np.array([extent[0], extent[1], extent[0], extent[1]])
testImagePlotT = axsT[0].imshow(truePlate, cmap = cmap, extent = extentPlot)
axsT[0].set_xlabel('x [mm]', fontsize = 14)
axsT[0].set_ylabel('y [mm]', fontsize = 14)
figT.colorbar(testImagePlotT, ax = axsT[0], orientation = 'horizontal')

# Plotting grouped averaged image
cropPlate = truePlate[int(truePlate.shape[0]/2 - 100) : int(truePlate.shape[0]/2 + 100),
                      int(truePlate.shape[0]/2 - 100) : int(truePlate.shape[0]/2 + 100)]
cropExtent = extentPlot * cropPlate.shape[1]/truePlate.shape[1]
resultPlotT = axsT[1].imshow(cropPlate, cmap = cmap, extent = cropExtent)

axsT[1].set_xlabel('x [mm]', fontsize = 14)
figT.colorbar(resultPlotT, ax = axsT[1], orientation = 'horizontal')
figT.tight_layout()

plt.show()

In [ ]:
# ––– Plotting the Characteristic Speckle Analysis –––
from PlottingTools import AutoCorr
_, _, _, _ = AutoCorr(np.sqrt(outputFP), extent = extent, truncRadius = targetRadius, title = "Characteristic Speckle")